In [ ]:
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3, VGG16

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

import os
import requests
import zipfile
import glob

from sklearn.model_selection import train_test_split

from os import listdir
from os.path import isdir, isfile, join

In [ ]:
target_directory = 'data/'

zip_url = 'https://www.muratkoklu.com/datasets/vtdhnd10.php'

os.makedirs(target_directory, exist_ok=True)
response = requests.get(zip_url)
zip_path = os.path.join(target_directory, 'data.zip')

with open(zip_path, 'wb') as zip_file:
    zip_file.write(response.content)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_directory)

In [ ]:
data_dir = os.path.join(target_directory, 'Grapevine_Leaves_Image_Dataset/*')

data = []
labels = []


for class_folder in glob.glob(data_dir):
    label = os.path.basename(class_folder)

    for image_path in glob.glob(os.path.join(class_folder, '*.png')):
        labels.append(label)
        data.append(image_path)

In [ ]:
print(len(data))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [ ]:
print(X_train)

In [ ]:
img_width = 300
img_height = 300

EPOCHS = 20
BATCH_SIZE = 32

IMAGE_SHAPE = (img_width, img_height)

In [ ]:
datagen = ImageDataGenerator(
    rescale=1/255.,
    validation_split=0.1
)

In [ ]:
train_gen = datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_train, 'label': y_train}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical',
    shuffle=True
)

validation_gen = datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_train, 'label': y_train}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


test_gen = datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_test, 'label': y_test}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SHAPE,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

In [ ]:
CLASSES = set(y_train)

# BASIC SEQUENTIAL MODEL

In [ ]:
model = Sequential()

# Add a 2D convolutional layer with 32 filters of size 3x3 and ReLU activation function
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))

# Add a MaxPooling layer to reduce the data size
model.add(MaxPooling2D((2, 2)))

# Add another 2D convolutional layer with 64 filters of size 3x3 and ReLU activation
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another MaxPooling layer
model.add(MaxPooling2D((2, 2)))

# Add a 2D convolutional layer with 128 filters of size 3x3 and ReLU activation
model.add(Conv2D(128, (3, 3), activation='relu'))

# Add another MaxPooling layer
model.add(MaxPooling2D((2, 2)))

# Flatten the data into a one-dimensional array
model.add(Flatten())

# Add a dense layer with 512 units and ReLU activation
model.add(Dense(512, activation='relu'))

# Add an output layer with one neuron and a sigmoid activation function (binary classification)
model.add(Dense(len(CLASSES), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    epochs=EPOCHS,
    validation_data=validation_gen,
    validation_steps=len(validation_gen)
)

# STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
# STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

# history = model.fit_generator(generator=train_gen,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=validation_gen,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=EPOCHS
# )

In [ ]:
model.save('basic_model.keras')

# Model 1

### WITH DROPOUTS, BATCH_NORM AND EARLY STOPPING

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(CLASSES), activation='softmax'))

In [ ]:
# Compile the model
opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

history = model.fit_generator(generator=train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=EPOCHS,
                    callbacks=[callback],
                    verbose=1
)

In [ ]:
model.save('model1.keras')

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(CLASSES), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

# Model 2

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    epochs=EPOCHS,
    validation_data=validation_gen,
    validation_steps=len(validation_gen)
)